In [34]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

In [35]:
max_pages = 100
max_available_pages = None
pages = 1
# index=['site', 'date', 'stars', 'title', 'text', 'location']
result_df = pd.DataFrame()

website = 'www.apple.com'

response = requests.get(f'https://www.trustpilot.com/review/' + website)
web_page = response.text
soup = BeautifulSoup(web_page, 'lxml')

max_available_pages = int(soup.find('a', {'name': 'pagination-button-last'}).find('span').string)


In [36]:
# result_df = pd.DataFrame()

In [37]:
def get_reviews_from_page(page):
    return_df = pd.DataFrame()
    articles = page.find_all('article')

    for article in articles:
        country = article.find_next('aside').find_all('span')[-1].string

        article_section = article.find_next('section')
        stars = re.search(r'\d+', article_section.find('img')['alt']).group()
        date = article_section.find('time')['datetime']
        title = article_section.find('h2').string
        text_raw = article_section.find('p').contents
        text = ' '.join(line.text for line in text_raw if line.text != '')

        return_df = pd.concat([return_df , pd.DataFrame([{'site': website, 'date': date, 'stars': stars, 'title': title, 'text': text, 'location': country}])], ignore_index=True)

    return return_df

In [38]:
# already fetched site - no need for duplicate fetching
result_df = pd.concat([result_df, get_reviews_from_page(soup)], ignore_index=True)

for index in range(2, max_available_pages if max_pages > max_available_pages else max_pages):
    page = BeautifulSoup(requests.get(f'https://www.trustpilot.com/review/' + website + "?page=" + str(index)).text, 'lxml')
    result_df = pd.concat([result_df, get_reviews_from_page(page)], ignore_index=True)



In [41]:
result_df.to_csv("data/" + website + ' - Trustpilot.csv')